In [113]:
import simplejson as json  
import os
import lzma
import re
import pandas as pd

In [4]:
#setting directory
os.chdir('/Users/samk/Box/Case Law Project/UNH_MS_Work-master/data/raw/Vermont-20180905-text/data')

In [5]:
#Loading data
cases = []
 
with lzma.open('data.jsonl.xz') as infile:
    for line in infile:
        #decode the file into a convenient format
        record = json.loads(str(line, 'utf-8'))
        #if the decision date on the case matches one we're interested in, add to our list
        cases.append(record)

print("Number of Cases: {}".format(len(cases)))


Number of Cases: 21578


In [105]:
#Extracting meta data and text of cases
def parse_cases(case):
    
    elem_dict = {}
    try:
        elem_dict["id"] = case['id']
    except IndexError:
        elem_dict["id"] = 'null'
    try:
        elem_dict["name"] = case['name']
    except IndexError:
        elem_dict["name"] = 'null'
    try:   
        elem_dict["name_abbreviation"] = case['name_abbreviation']
    except IndexError:
        elem_dict["name_abbreviation"] = 'null'
    try:
        elem_dict["parties"] = case['casebody']['data']['parties']
    except IndexError:
         elem_dict["parties"] = 'null'
    try:
        elem_dict["plaintiff"] = case['name_abbreviation'].split('v.')[0].strip()
    except IndexError:
        elem_dict["plaintiff"] = 'null'
    try:
        elem_dict["defendant"] = case['name_abbreviation'].split('v.')[1].strip()
    except IndexError:
        elem_dict["defendant"] = 'null'
    try:
        elem_dict["decision_date"] = case['decision_date']
    except IndexError:
        elem_dict["decision_date"] = 'null'
    try:
        elem_dict["docket_number"] = case['docket_number']
    except IndexError:
        elem_dict["docket_number"] = 'null'
        
    citation = [citation for citation in case['citations'] if citation['type'] == 'official'][0]
    elem_dict["citation_type"] = citation['type']
    elem_dict["citation_num"] = citation['cite']
    try:
        elem_dict["reporter"] = case['reporter']['full_name']
    except IndexError:
        elem_dict["reporter"] = 'null'
    try:
        elem_dict["court_id"] = case['court']['id']
    except IndexError:
        elem_dict["court_id"] = 'null'
        
    elem_dict["court_name"] = case['court']['name']
    elem_dict["jurisdiction_id"] = case['jurisdiction']['id']
    elem_dict["jurisdiction_name"] = case['jurisdiction']['name_long']
    try:
        elem_dict["plaintiff_attorneys"] = case['casebody']['data']['attorneys'][0]
    except IndexError:
        elem_dict["plaintiff_attorneys"] = 'null'
    try:
        elem_dict["defendants_attorneys"] = case['casebody']['data']['attorneys'][1]
    except IndexError:
        elem_dict["defendants_attorneys"] = 'null'
    elem_dict["judges"] = case['casebody']['data']['judges']
    opinion_type = [opinion['type'] for opinion in case['casebody']['data']['opinions']]
    elem_dict["opinion_type"]= opinion_type
    opinion_author = [opinion['author'] for opinion in case['casebody']['data']['opinions']]
    elem_dict["opinion_author"] = opinion_author
    elem_dict["head_matter"] = case['casebody']['data']['head_matter']
    opinion_text = [opinion['text'] for opinion in case['casebody']['data']['opinions']]
    elem_dict["opinion_text"] = opinion_text
    
    return elem_dict



In [109]:
#parsing the cases
parsed_cases = []
for case in cases:
    doc = parse_cases(case)
    parsed_cases.append(doc)

In [117]:
#Dumping Cases in Pandas DF
cases_df = pd.DataFrame(parsed_cases)
cases_df

,citation_num,citation_type,court_id,court_name,decision_date,defendant,defendants_attorneys,docket_number,head_matter,id,...,jurisdiction_name,name,name_abbreviation,opinion_author,opinion_text,opinion_type,parties,plaintiff,plaintiff_attorneys,reporter
0,169 Vt. 426,official,8945,Vermont Supreme Court,1999-06-25,Glens Falls Insurance,"William, F. Ellis and Kevin J. Coyle of McNeil...",No. 98-073,No. 98-073\n[736 A.2d 768]\nAgency of Natural ...,174216,...,Vermont,"Agency of Natural Resources, State of Vermont ...",Agency of Natural Resources v. Glens Falls Ins...,"[Johnson, J.]","[Johnson, J.\nThis case concerns a dispute amo...",[majority],"[Agency of Natural Resources, State of Vermont...",Agency of Natural Resources,"William, H. Sorrell, Attorney General, and Wil...",Vermont Reports
1,169 Vt. 62,official,8945,Vermont Supreme Court,1999-02-05,Koveos,"Karen Shingler, Burlington, and Nathan Z. Ders...",No. 98-248,No. 98-248\n[732 A.2d 722]\nState of Vermont v...,174219,...,Vermont,State of Vermont v. Emmanuel E. Koveos,State v. Koveos,"[Dooley, J.]","[Dooley, J.\nDefendant appeals from a judgment...",[majority],[State of Vermont v. Emmanuel E. Koveos],State,"Lauren Bowerman, Chittenden County State’s Att...",Vermont Reports
2,169 Vt. 604,official,8945,Vermont Supreme Court,1999-07-13,Jewell,null,No. 98-092,No. 98-092\n[737 A.2d 897]\nIn re Appeal of Ma...,174220,...,Vermont,In re Appeal of Max E. JEWELL and Judith Belye...,Town of Hartford v. Jewell,[None],[The Town of Hartford appeals from an environm...,[majority],[In re Appeal of Max E. JEWELL and Judith Bely...,Town of Hartford,null,Vermont Reports
3,169 Vt. 74,official,8945,Vermont Supreme Court,1998-12-18,Town of Milton,"William H. Sorrell, Attorney General, Ron Shem...",No. 97-422,"No. 97-422\n[726 A.2d 477]\nWilbur Parker, et ...",174224,...,Vermont,"Wilbur Parker, et al. v. Town of Milton, et al.",Parker v. Town of Milton,"[Johnson, J.]","[Johnson, J.\nPlaintiffs, six individual Vermo...",[majority],"[Wilbur Parker, et al. v. Town of Milton, et al.]",Parker,"John L. Franco, Jr., Burlington, for Plaintiff...",Vermont Reports
4,169 Vt. 625,official,8945,Vermont Supreme Court,1999-08-02,Williams,null,No. 98-492,No. 98-492\n[740 A.2d 791]\nJames MURRAY v. Ma...,174227,...,Vermont,James MURRAY v. Mary Louise WILLIAMS,Murray v. Williams,[None],[Defendant Mary Louise Williams appeals the Ch...,[majority],[James MURRAY v. Mary Louise WILLIAMS],Murray,null,Vermont Reports
5,169 Vt. 364,official,8945,Vermont Supreme Court,1999-06-25,Leas,"Susan M. Murray of Langrock Sperry & Wool, Mid...",No. 97-452,No. 97-452\n[737 A.2d 889]\nSandra Felice Leas...,174230,...,Vermont,Sandra Felice Leas v. James Marc Leas,Leas v. Leas,"[Dooley, J.]","[Dooley, J.\nFather appeals from a Chittenden ...",[majority],[Sandra Felice Leas v. James Marc Leas],Leas,"John J. Bergeron of Bergeron, Paradis, Fitzpat...",Vermont Reports
6,169 Vt. 476,official,8945,Vermont Supreme Court,1999-08-27,null,"Leslie A. Cadwell, Montpelier, for Appellee.",No. 98-332,No. 98-332\n[739 A.2d 671]\nIn re Investigatio...,174233,...,Vermont,In re Investigation Into the Existing Rates of...,In re Investigation Into the Existing Rates of...,"[Amestoy, C.J.]","[Amestoy, C.J.\nAppellant Vermont Telephone Co...",[majority],[In re Investigation Into the Existing Rates o...,In re Investigation Into the Existing Rates of...,"Michael Marks of Tarrant, Marks & Gillies, Mon...",Vermont Reports
7,169 Vt. 572,official,8945,Vermont Supreme Court,1999-04-08,Roya,null,No. 97-471,No. 97-471\n[730 A.2d 600]\nSTATE of Vermont v...,174236,...,Vermont,STATE of Vermont v. Kenneth G. ROYA,State v. Roya,[None],"[State of Vermont appeals the November 11, 199...",[majority],[STATE of Vermont v. Kenneth G. ROYA],State,null,Vermont Reports
8,169 Vt. 630,official,8945,Vermont Supreme Court,1999-08-18,Bennett,null,No. 99-240,"No. 99-240\n[739 A.2d 1210]\nArthur STOWELL, S...",174238,...,Vermont,"Arthur STOWELL, Sr. v. Sheila BENNETT",Stowell v. Bennett